### LGBM을 이용해 임대보증금 / 임대료 결측치 처리해보기

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/train.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/test.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [3]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수']

In [4]:
train['주차면수/총세대수'] = train['단지내주차면수']/train['총세대수']
train['공가수/주차면수'] = train['공가수']/train['단지내주차면수']
train['총임대세대수'] = train.groupby('단지코드')['전용면적별세대수'].transform(sum)

train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0,0.694561,0.042169,239
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239


In [5]:
test['주차면수/총세대수'] = test['단지내주차면수']/test['총세대수']
test['공가수/주차면수'] = test['공가수']/test['단지내주차면수']
test['총임대세대수'] = test.groupby('단지코드')['전용면적별세대수'].transform(sum)

test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,0.905836,0.020498,754
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,0.905836,0.020498,754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.081370,670
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0,0.785340,0.150000,378


### 오류 처리

In [6]:
drop_danji = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 
              'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']    # 총 12개

for code in drop_danji:
    train = train[train['단지코드'] != code]
    
train = train.reset_index(drop=True)

### 지하철 결측치 처리

In [7]:
train['지하철'] = train['지하철'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

### 버스정류장 결측치 처리

In [8]:
train['버스정류장'] = train['버스정류장'].fillna(2)

### 자격유형 결측치 처리

In [9]:
test.loc[196, '자격유형'] = 'A'
test.loc[258, '자격유형'] = 'C'

### QCUT

In [10]:
train['qcut_총세대수'] = pd.qcut(train['총세대수'], 5, labels=False)
train['qcut_단지내주차면수'] = pd.qcut(train['단지내주차면수'], 5, labels=False)
train['qcut_전용면적별세대수'] = pd.qcut(train['전용면적별세대수'], 5, labels=False)

In [11]:
test['qcut_총세대수'] = pd.qcut(test['총세대수'], 5, labels=False)
test['qcut_단지내주차면수'] = pd.qcut(test['단지내주차면수'], 5, labels=False)
test['qcut_전용면적별세대수'] = pd.qcut(test['전용면적별세대수'], 5, labels=False)

In [12]:
train.isnull().sum(), test.isnull().sum()

(단지코드               0
 총세대수               0
 임대건물구분             0
 지역                 0
 공급유형               0
 전용면적               0
 전용면적별세대수           0
 공가수                0
 자격유형               0
 임대보증금            569
 임대료              569
 지하철                0
 버스정류장              0
 단지내주차면수            0
 등록차량수              0
 주차면수/총세대수          0
 공가수/주차면수           0
 총임대세대수             0
 qcut_총세대수          0
 qcut_단지내주차면수       0
 qcut_전용면적별세대수      0
 dtype: int64,
 단지코드               0
 총세대수               0
 임대건물구분             0
 지역                 0
 공급유형               0
 전용면적               0
 전용면적별세대수           0
 공가수                0
 자격유형               0
 임대보증금            180
 임대료              180
 지하철                0
 버스정류장              0
 단지내주차면수            0
 주차면수/총세대수          0
 공가수/주차면수           0
 총임대세대수             0
 qcut_총세대수          0
 qcut_단지내주차면수       0
 qcut_전용면적별세대수      0
 dtype: int64)

### 면적 10단위로 끊기

In [13]:
train['10_전용면적'] = np.round(train['전용면적'], -1)
train.loc[train['10_전용면적']>=100, '10_전용면적'] = 100

train['10_전용면적'].value_counts()

50.0     1007
40.0      697
30.0      583
20.0      223
60.0      203
80.0       68
70.0       52
100.0      28
10.0        8
Name: 10_전용면적, dtype: int64

In [14]:
test['10_전용면적'] = np.round(test['전용면적'], -1)
test.loc[test['10_전용면적']>=100, '10_전용면적'] = 100

test['10_전용면적'].value_counts()

50.0     348
40.0     287
30.0     173
20.0     102
60.0      68
80.0      18
10.0       9
100.0      9
70.0       8
Name: 10_전용면적, dtype: int64

In [15]:
codes = train['단지코드'].value_counts().index

for code in codes:
    for area in range(10, 101, 10):
        train_bool = (train['10_전용면적']==area)&(train['단지코드']==code)
        train.loc[train_bool, f'{area}평_비율'] = train.loc[train_bool, '전용면적별세대수'] / train.loc[train_bool, '총임대세대수']

In [16]:
for area in range(10, 101, 10):
    train[f'{area}평_비율'] = train[f'{area}평_비율'].fillna(0)
    train[f'{area}평_비율'] = train.groupby('단지코드')[f'{area}평_비율'].transform(sum)

In [17]:
codes = test['단지코드'].value_counts().index

for code in codes:
    for area in range(10, 101, 10):
        test_bool = (test['10_전용면적']==area)&(test['단지코드']==code)
        test.loc[test_bool, f'{area}평_비율'] = test.loc[test_bool, '전용면적별세대수'] / test.loc[test_bool, '총임대세대수']

In [18]:
for area in range(10, 101, 10):
    test[f'{area}평_비율'] = test[f'{area}평_비율'].fillna(0)
    test[f'{area}평_비율'] = test.groupby('단지코드')[f'{area}평_비율'].transform(sum)

### Age 정보 넣기

In [19]:
## train1, test1 : 라벨인코딩 + age정보 같이 된 데이터프레임
train1 = train.copy()
test1 = test.copy()

age.index = age['지역']
age.drop('지역', axis=1, inplace=True)

In [20]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])

for i in range(train.shape[0]):
    area = train.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
    
age_df

,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
3,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
4,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2865,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2866,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2867,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


In [21]:
train1 = pd.concat([train1, age_df], axis=1)
train1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,4,30.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,2,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,3,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


In [22]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])

for i in range(test1.shape[0]):
    area = test1.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
    
age_df

,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
2,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
3,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1018,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1019,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1020,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


In [23]:
test1 = pd.concat([test1, age_df], axis=1)
test1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,40.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,1,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,0.905836,0.020498,754,2,3,2,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.081370,670,2,2,3,40.0,0.0,0.432836,0.110448,0.188060,0.268657,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378,0,1,3,30.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378,0,1,1,30.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0,0.785340,0.150000,378,0,1,4,40.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


### < 임대보증금 / 임대료 >

#### 임대료 " - " 항목

- 서울특별시 - 장기전세 : 없는게 정상 -> 0으로 채우기

- 부산광역시 - 국민임대 - (H) :
    - A와 시세가 비슷함 -> 평균내서 채우기
    - 머신러닝 모델을 이용한 예측 -> 지역을 제외시키고 A와 H에 대해 One-Hot 인코딩 진행
    - 경기도 : A = 380가구, H = 97가구
   
- 강원도 - 행복주택 - (K) : 
    - 행복주택 + K 인 표본 : 충청북도 13가구, 국민임대 123가구 / 경상남도 16가구, 국민임대 202가구
    
- 대구광역시 - 국민임대 - (H) : 3결측, 1결측X
    - 결측치가 아닌 A 모델이 75가구 존재, 부산광역시의 경우와 같이 평균을 내던지 학습을 통한 예측을 해야함
    
---
    
#### 임대료 결측치 항목

- 대전광역시 - 공공분양 : 유일한 아파트 결측치

- 상가 - 임대상가 - (D) : 
    - 전용면적별세대수 = 1 : 전부다 1의 값을 가지기 때문에 방해요소가 될수도 있음
        - 부산광역시 : 220가구 (8개 단지)
        - 대전광역시 : 120가구 (7개 단지)
        - 경상남도 : 97가구 (8개 단지)
        - 충청남도 : 76가구 (5개 단지)
        - 강원도 : 31가구 (4개 단지)
        - 제주특별자치도 : 25가구 (2개 단지)
    - 상가에 대한 수치는 힌트가 없기 때문에 일반적인 수치로 채워넣는 것이 좋을 수 있음

---

- 서울특별시 - 장기전세 : 없는게 정상 -> 0으로 채우기

In [25]:
train[train['단지코드']=='C1039']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
2073,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,0,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,3,50.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2074,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,0,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2,60.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2075,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,0,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2,60.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2076,C1039,790,아파트,서울특별시,국민임대,26.68,26,13.0,A,15282000,190620,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,1,30.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2077,C1039,790,아파트,서울특별시,국민임대,36.70,140,13.0,A,21024000,256040,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,3,40.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2078,C1039,790,아파트,서울특별시,국민임대,36.70,14,13.0,A,21024000,256040,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,1,40.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2079,C1039,790,아파트,서울특별시,국민임대,36.87,43,13.0,A,21024000,256040,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2,40.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2080,C1039,790,아파트,서울특별시,국민임대,46.76,192,13.0,A,45456000,356420,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,4,50.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2081,C1039,790,아파트,서울특별시,국민임대,46.96,25,13.0,A,45456000,356420,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,1,50.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0
2082,C1039,790,아파트,서울특별시,영구임대,26.68,70,13.0,I,11490000,94820,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2,30.0,0.0,0.0,0.159494,0.249367,0.434177,0.156962,0.0,0.0,0.0,0.0


In [24]:
train.loc[2073:2075, '임대료'] = 0

---

- 부산광역시 - 국민임대 - (H) :
    - A와 시세가 비슷함 -> 평균내서 채우기
    - 머신러닝 모델을 이용한 예측 -> 지역을 제외시키고 A와 H에 대해 One-Hot 인코딩 진행
    - 경기도 : A = 380가구, H = 97가구

#### 결측치 dataframe을 따로 나눔

In [26]:
pd.set_option('display.max_rows', 30)
train.groupby('지역')['자격유형'].value_counts()

지역       자격유형
강원도      A       136
         D        31
         C         9
         H         4
         K         4
         E         1
         I         1
         J         1
경기도      A       400
         H        97
         J        25
         I        23
         E        12
         L        12
         N        12
         G         8
         B         6
경상남도     A       205
         D        97
         C        20
         K        16
         H        12
         L         8
         I         3
         E         2
경상북도     A       100
         J         5
         E         3
         H         3
광주광역시    A       103
         N        10
         E         9
         J         8
         L         6
         I         2
         M         2
         O         1
대구광역시    A        75
         B        10
         J         9
         C         4
         H         4
대전광역시    D       120
         A       109
         C        23
         I         8
         J         5

In [44]:
train_na = train1[(train1['임대료'].isna())|(train1['임대료']=='-')]
train_notna = train1[~((train1['임대료'].isna())|(train1['임대료']=='-'))]

train_na.shape, train_notna.shape

((581, 54), (2288, 54))

In [45]:
test_na = test1[(test1['임대료'].isna())|(test1['임대료']=='-')]
test_notna = test1[~((test1['임대료'].isna())|(test1['임대료']=='-'))]

test_na.shape, test_notna.shape

((186, 53), (836, 53))

In [30]:
train_na.groupby('지역')['자격유형'].value_counts()

지역       자격유형
강원도      D        31
         K         4
경상남도     D        97
대구광역시    H         3
대전광역시    D       120
부산광역시    D       220
         H         5
제주특별자치도  D        25
충청남도     D        76
Name: 자격유형, dtype: int64

X_train : 결측치가 없는 A, H 자격유형 데이터

y_train : 결측치가 없는 임대보증금 / 임대료 데이터

X_test : 결측치가 있는 A, H 자격유형 데이터

y_test : 결측치가 있는 임대보증금 / 임대료 데이터

In [47]:
X_train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '등록차량수', '주차면수/총세대수',
       '공가수/주차면수', '총임대세대수', 'qcut_총세대수', 'qcut_단지내주차면수', 'qcut_전용면적별세대수',
       '10_전용면적', '10평_비율', '20평_비율', '30평_비율', '40평_비율', '50평_비율', '60평_비율',
       '70평_비율', '80평_비율', '90평_비율', '100평_비율', '10대미만(여자)', '10대미만(남자)',
       '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)',
       '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)',
       '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)',
       '100대(여자)', '100대(남자)'],
      dtype='object')

In [153]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '공급유형', '전용면적', '전용면적별세대수', '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)', '자격유형_Type', '지역']

X = train1[~((train1['임대보증금']=='-')|(train1['임대보증금'].isna()))&((train1['자격유형']=='A')|(train1['자격유형']=='H'))].copy().reset_index(drop=True)
test_X = train1[((train1['임대보증금']=='-')|(train1['임대보증금'].isna()))&((train1['자격유형']=='A')|(train1['자격유형']=='H'))].copy().reset_index(drop=True)

X.loc[X['자격유형']=='A', '자격유형_Type'] = 0
X.loc[X['자격유형']=='H', '자격유형_Type'] = 1
test_X.loc[test_X['자격유형']=='A', '자격유형_Type'] = 0
test_X.loc[test_X['자격유형']=='H', '자격유형_Type'] = 1

encoder = LabelEncoder()
encoder.fit(X['공급유형'])
X['공급유형'] = encoder.transform(X['공급유형'])
test_X['공급유형'] = encoder.transform(test_X['공급유형'])
print(encoder.classes_)

encoder.fit(X['지역'])
X['지역'] = encoder.transform(X['지역'])
test_X['지역'] = encoder.transform(test_X['지역'])
print(encoder.classes_)

X_train = X[sel]
y_train = X['임대료']
X_test = test_X[sel]

['공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '장기전세']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']


In [154]:
## 다른 지역의 임대보증금 / 임대료 데이터를 실수형으로 변환
## 그 후 로그를 씌워줌
y_train = y_train.astype('float')
# y_train = y_train.astype('float')

y_train = np.log1p(y_train)
# y_train = np.log1p(y_train)

In [155]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [156]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

score = cross_val_score(LGBM, X_train, y_train, cv=5, 
                       scoring='neg_mean_absolute_error')

m_score = round(np.abs(score.mean()), 6)
print('LightGBM Score :', m_score)

pred = LGBM.predict(X_test)
pred[0:10]

LightGBM Score : 0.313692


array([12.18835464, 12.10060519, 12.23364143, 12.31771911, 12.61256857,
       11.87512443, 11.98887894, 12.12927451])

### 임대보증금

#### 지역을 포함 안한 예측치

In [131]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C1326,-,-,1.269298e+07
1,C1326,-,-,1.351534e+07
2,C1326,-,-,1.271817e+07
3,C1326,-,-,2.668761e+07
4,C1326,-,-,3.328574e+07
5,C2186,-,-,9.784373e+06
6,C2186,-,-,9.768140e+06
7,C2186,-,-,1.772133e+07


#### 지역을 포함한 예측치

In [151]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C1326,-,-,1.142800e+07
1,C1326,-,-,1.258449e+07
2,C1326,-,-,1.154101e+07
3,C1326,-,-,2.579061e+07
4,C1326,-,-,3.322497e+07
5,C2186,-,-,9.860035e+06
6,C2186,-,-,9.901309e+06
7,C2186,-,-,1.775785e+07


#### 결측치 채우기

In [152]:
null_idx = train[(train['임대료']=='-')&(train['자격유형']=='H')].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대보증금'] = one
    
train[(train['임대료']=='-')&(train['자격유형']=='H')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
2475,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,1.1428e+07,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,4,20.0,0.0,0.297828,0.305067,0.239917,0.157187,0.0,0.0,0.0,0.0,0.0
2476,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.0,H,1.25845e+07,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,3,20.0,0.0,0.297828,0.305067,0.239917,0.157187,0.0,0.0,0.0,0.0,0.0
2477,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.0,H,1.1541e+07,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,4,30.0,0.0,0.297828,0.305067,0.239917,0.157187,0.0,0.0,0.0,0.0,0.0
2478,C1326,1934,아파트,부산광역시,국민임대,37.70,464,43.0,H,2.57906e+07,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,4,40.0,0.0,0.297828,0.305067,0.239917,0.157187,0.0,0.0,0.0,0.0,0.0
2479,C1326,1934,아파트,부산광역시,국민임대,46.94,304,43.0,H,3.3225e+07,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,4,50.0,0.0,0.297828,0.305067,0.239917,0.157187,0.0,0.0,0.0,0.0,0.0
2823,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.0,H,9.86004e+06,-,0.0,8.0,664.0,744.0,0.718615,0.000000,916,3,3,4,30.0,0.0,0.000000,0.554585,0.240175,0.205240,0.0,0.0,0.0,0.0,0.0
2824,C2186,924,아파트,대구광역시,국민임대,29.34,38,0.0,H,9.90131e+06,-,0.0,8.0,664.0,744.0,0.718615,0.000000,916,3,3,1,30.0,0.0,0.000000,0.554585,0.240175,0.205240,0.0,0.0,0.0,0.0,0.0
2825,C2186,924,아파트,대구광역시,국민임대,37.43,220,0.0,H,1.77578e+07,-,0.0,8.0,664.0,744.0,0.718615,0.000000,916,3,3,4,40.0,0.0,0.000000,0.554585,0.240175,0.205240,0.0,0.0,0.0,0.0,0.0


### 임대료

In [157]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대료
0,C1326,-,-,196486.607490
1,C1326,-,-,179979.751278
2,C1326,-,-,205589.463574
3,C1326,-,-,223622.500296
4,C1326,-,-,300308.403914
5,C2186,-,-,143647.472554
6,C2186,-,-,160953.813273
7,C2186,-,-,185214.355739


In [158]:
null_idx = train[(train['임대료']=='-')&(train['자격유형']=='H')].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대료'] = one
    
train[(train['임대료']=='-')&(train['자격유형']=='H')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율


---

- 강원도 - 행복주택 - (K) : 
    - 행복주택 + K 인 표본 : 충청북도 13가구, 국민임대 123가구 / 경상남도 16가구, 국민임대 202가구

X_train : 충북, 경남의 국민임대, 행복주택 정보

y_train : 충북, 경남의 국민임대, 행복주택 임대보증금 / 임대료

X_test : 강원도의 국민임대, 행복주택 정보

y_test : 강원도의 국민임대, 행복주택 임대보증금 / 임대료

In [199]:
X = train1[((train1['지역']=='충청북도')|(train1['지역']=='경상남도'))&\
           (((train1['공급유형']=='국민임대')&(train1['자격유형']=='A'))|((train1['공급유형']=='행복주택')&\
                                                              (train1['자격유형']=='K')))].copy().reset_index(drop=True)   # 충북, 경남의 국민임대, 행복주택 데이터

test_X = train1[(train1['지역']=='강원도')&\
                ((train1['공급유형']=='행복주택')&(train1['자격유형']=='K'))].copy().reset_index(drop=True)   # 강원도의 국민임대, 행복주택 데이터

In [200]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '공급유형', '전용면적', '전용면적별세대수', '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)']

encoder = LabelEncoder()
encoder.fit(X['공급유형'])
X['공급유형'] = encoder.transform(X['공급유형'])
test_X['공급유형'] = encoder.transform(test_X['공급유형'])
print(encoder.classes_)

X_train = X[sel]
y_train = X['임대료']
X_test = test_X[sel]

['국민임대' '행복주택']


In [201]:
## 다른 지역의 임대보증금 / 임대료 데이터를 실수형으로 변환
## 그 후 로그를 씌워줌
y_train = y_train.astype('float')
# y_train = y_train.astype('float')

y_train = np.log1p(y_train)
# y_train = np.log1p(y_train)

In [185]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [202]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

score = cross_val_score(LGBM, X_train, y_train, cv=5, 
                       scoring='neg_mean_absolute_error')

m_score = round(np.abs(score.mean()), 6)
print('LightGBM Score :', m_score)

pred = LGBM.predict(X_test)
pred[0:10]

LightGBM Score : 0.378559


array([9.76975594, 9.76221227, 9.80641711, 9.93532377])

In [196]:
test_X

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1786,480,아파트,강원도,1,16.91,156,25.0,K,-,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,20.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.03276,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
1,C1786,480,아파트,강원도,1,26.90,136,25.0,K,-,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.03276,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2,C1786,480,아파트,강원도,1,26.90,72,25.0,K,-,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,2,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.03276,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
3,C1786,480,아파트,강원도,1,26.90,24,25.0,K,-,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,1,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.03276,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


In [194]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C1786,-,-,4.270188e+07
1,C1786,-,-,4.258043e+07
2,C1786,-,-,4.591636e+07
3,C1786,-,-,4.937826e+07


### 임대보증금

In [197]:
null_idx = train[(train['지역']=='강원도')&\
                ((train['공급유형']=='행복주택')&(train['자격유형']=='K'))].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대보증금'] = one
    
train[(train['지역']=='강원도')&\
                ((train['공급유형']=='행복주택')&(train['자격유형']=='K'))]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
2608,C1786,480,아파트,강원도,행복주택,16.91,156,25.0,K,4.27019e+07,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,20.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2609,C1786,480,아파트,강원도,행복주택,26.90,136,25.0,K,4.25804e+07,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2610,C1786,480,아파트,강원도,행복주택,26.90,72,25.0,K,4.59164e+07,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,2,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2611,C1786,480,아파트,강원도,행복주택,26.90,24,25.0,K,4.93783e+07,-,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,1,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0


### 임대료

In [204]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대료
0,C1786,-,-,17495.496544
1,C1786,-,-,17364.005377
2,C1786,-,-,18148.841540
3,C1786,-,-,20645.968290


In [205]:
null_idx = train[(train['지역']=='강원도')&\
                ((train['공급유형']=='행복주택')&(train['자격유형']=='K'))].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대료'] = one
    
train[(train['지역']=='강원도')&\
                ((train['공급유형']=='행복주택')&(train['자격유형']=='K'))]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
2608,C1786,480,아파트,강원도,행복주택,16.91,156,25.0,K,4.27019e+07,17495.5,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,20.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2609,C1786,480,아파트,강원도,행복주택,26.90,136,25.0,K,4.25804e+07,17364,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,3,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2610,C1786,480,아파트,강원도,행복주택,26.90,72,25.0,K,4.59164e+07,18148.8,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,2,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0
2611,C1786,480,아파트,강원도,행복주택,26.90,24,25.0,K,4.93783e+07,20646,0.0,3.0,338.0,345.0,0.704167,0.073964,480,0,1,1,30.0,0.0,0.325,0.483333,0.191667,0.0,0.0,0.0,0.0,0.0,0.0


### '-' 결측치 모두 처리

In [206]:
train[train['임대료']=='-']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율


---

## 상가 결측치 처리

In [207]:
train['자격유형'].value_counts()

A    1729
D     569
H     155
J     103
C      95
I      49
E      37
L      33
K      33
N      30
B      21
G       9
F       3
M       2
O       1
Name: 자격유형, dtype: int64

In [209]:
train[train['공급유형']=='공공분양']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
2284,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,4,70.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2285,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,3,70.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2286,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1,70.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2287,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,3,80.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2288,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1,80.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2289,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1,80.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0
2290,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1,80.0,0.0,0.0,0.0,0.0,0.179872,0.312634,0.339757,0.167737,0.0,0.0


In [335]:
X = train1[train1['자격유형']=='A'].copy().reset_index(drop=True)

test_X = train1[train1['자격유형']=='D'].copy().reset_index(drop=True)

In [336]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '지역', '전용면적', '전용면적별세대수', '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)']

encoder = LabelEncoder()
encoder.fit(X['지역'])
X['지역'] = encoder.transform(X['지역'])
test_X['지역'] = encoder.transform(test_X['지역'])
print(encoder.classes_)

X_train = X[sel]
y_train = X['임대보증금']
X_test = test_X[sel]

['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']


In [337]:
## 다른 지역의 임대보증금 / 임대료 데이터를 실수형으로 변환
## 그 후 로그를 씌워줌
y_train = y_train.astype('float')
# y_train = y_train.astype('float')

y_train = np.log1p(y_train)
# y_train = np.log1p(y_train)

In [338]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [339]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

score = cross_val_score(LGBM, X_train, y_train, cv=5, 
                       scoring='neg_mean_absolute_error')

m_score = round(np.abs(score.mean()), 6)
print('LightGBM Score :', m_score)

pred = LGBM.predict(X_test)
pred[0:10]

LightGBM Score : 0.218203


array([15.94056607, 15.94056607, 15.94056607, 17.00940855, 15.68123726,
       15.68123726, 15.68123726, 15.68123726, 15.68123726, 15.68123726])

In [340]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C1925,NaN,NaN,8.373361e+06
1,C1925,NaN,NaN,8.373361e+06
2,C1925,NaN,NaN,8.373361e+06
3,C1925,NaN,NaN,2.438329e+07
4,C1874,NaN,NaN,6.460628e+06
...,...,...,...,...
564,C1350,NaN,NaN,5.259324e+07
565,C1350,NaN,NaN,6.891655e+07
566,C1350,NaN,NaN,6.056628e+07
567,C1350,NaN,NaN,6.417259e+07


### 임대보증금

In [341]:
null_idx = train[train['자격유형']=='D'].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대보증금'] = one
    
train.isnull().sum()

단지코드       0
총세대수       0
임대건물구분     0
지역         0
공급유형       0
          ..
60평_비율     0
70평_비율     0
80평_비율     0
90평_비율     0
100평_비율    0
Length: 32, dtype: int64

### 임대료

In [333]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대료
0,C1925,NaN,NaN,128349.410256
1,C1925,NaN,NaN,128349.410256
2,C1925,NaN,NaN,128349.410256
3,C1925,NaN,NaN,435449.637591
4,C1874,NaN,NaN,85512.794373
...,...,...,...,...
564,C1350,NaN,NaN,380778.939323
565,C1350,NaN,NaN,247766.852046
566,C1350,NaN,NaN,455124.117830
567,C1350,NaN,NaN,376389.341620


In [334]:
null_idx = train[train['자격유형']=='D'].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    train.loc[idx, '임대료'] = one
    
train.isnull().sum()

단지코드       0
총세대수       0
임대건물구분     0
지역         0
공급유형       0
          ..
60평_비율     0
70평_비율     0
80평_비율     0
90평_비율     0
100평_비율    0
Length: 32, dtype: int64

## test 결측치 처리

In [231]:
test[test['임대료'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,1,30.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,1,30.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,50.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
81,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
82,C1006,1505,상가,대전광역시,임대상가,37.26,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
83,C1006,1505,상가,대전광역시,임대상가,37.41,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
84,C1006,1505,상가,대전광역시,임대상가,37.41,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
85,C1006,1505,상가,대전광역시,임대상가,37.41,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664
86,C1006,1505,상가,대전광역시,임대상가,37.49,1,27.0,D,NaN,NaN,2.0,5.0,428.0,0.284385,0.063084,1505,4,2,0,40.0,0.000000,0.000000,0.982724,0.008638,0.007309,0.000000,0.0,0.000664,0.0,0.000664


In [235]:
train[(train['공급유형']=='영구임대')&(train['지역']=='대전광역시')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
145,C2621,1396,아파트,대전광역시,영구임대,26.37,890,26.0,C,5787000,79980,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,4,30.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
146,C2621,1396,아파트,대전광역시,영구임대,26.37,141,26.0,C,5787000,79980,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,3,30.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
147,C2621,1396,아파트,대전광역시,영구임대,26.37,20,26.0,C,5787000,79980,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,1,30.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
148,C2621,1396,아파트,대전광역시,영구임대,26.37,20,26.0,C,5787000,79980,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,1,30.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
149,C2621,1396,아파트,대전광역시,영구임대,31.32,298,26.0,C,6873000,94990,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,4,30.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
150,C2621,1396,아파트,대전광역시,영구임대,52.74,9,26.0,C,11574000,159960,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,1,50.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
151,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26.0,C,11574000,159960,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,1,50.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
152,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26.0,C,11574000,159960,1.0,4.0,277.0,133.0,0.198424,0.093863,1396,4,1,1,50.0,0.000000,0.000000,0.986390,0.000000,0.012178,0.000716,0.0,0.000000,0.0,0.000716
168,C1616,1507,아파트,대전광역시,영구임대,26.37,588,1.0,C,5787000,79980,0.0,2.0,407.0,98.0,0.270073,0.002457,1507,4,1,4,30.0,0.000000,0.000000,0.688786,0.307233,0.001991,0.000000,0.0,0.000664,0.0,0.001327
169,C1616,1507,아파트,대전광역시,영구임대,31.32,450,1.0,C,6873000,94990,0.0,2.0,407.0,98.0,0.270073,0.002457,1507,4,1,4,30.0,0.000000,0.000000,0.688786,0.307233,0.001991,0.000000,0.0,0.000664,0.0,0.001327


In [263]:
X = train1[(train1['공급유형']=='영구임대')&(train1['지역']=='대전광역시')]

test_X = test1[(test1['임대료'].isnull())&(test1['공급유형']=='영구임대')]

In [264]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '전용면적', '전용면적별세대수', '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)']

X_train = X[sel]
y_train = X['임대료']
X_test = test_X[sel]

In [265]:
## 다른 지역의 임대보증금 / 임대료 데이터를 실수형으로 변환
## 그 후 로그를 씌워줌
y_train = y_train.astype('float')
# y_train = y_train.astype('float')

y_train = np.log1p(y_train)
# y_train = np.log1p(y_train)

In [259]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [266]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

pred = LGBM.predict(X_test)
pred[0:10]

array([11.41554329, 11.41554329, 11.41554329])

In [261]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
76,C1006,NaN,NaN,7.944733e+06
77,C1006,NaN,NaN,7.944733e+06
79,C1006,NaN,NaN,7.944733e+06


### 임대보증금

In [262]:
null_idx = test1[(test1['임대료'].isnull())&(test1['공급유형']=='영구임대')].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대보증금'] = one
    
test.isnull().sum()

단지코드               0
총세대수               0
임대건물구분             0
지역                 0
공급유형               0
전용면적               0
전용면적별세대수           0
공가수                0
자격유형               0
임대보증금            177
임대료              180
지하철                0
버스정류장              0
단지내주차면수            0
주차면수/총세대수          0
공가수/주차면수           0
총임대세대수             0
qcut_총세대수          0
qcut_단지내주차면수       0
qcut_전용면적별세대수      0
10_전용면적            0
10평_비율             0
20평_비율             0
30평_비율             0
40평_비율             0
50평_비율             0
60평_비율             0
70평_비율             0
80평_비율             0
90평_비율             0
100평_비율            0
dtype: int64

### 임대료

In [267]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대료
76,C1006,NaN,NaN,90719.922539
77,C1006,NaN,NaN,90719.922539
79,C1006,NaN,NaN,90719.922539


In [268]:
null_idx = test1[(test1['임대료'].isnull())&(test1['공급유형']=='영구임대')].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대료'] = one
    
test.isnull().sum()

단지코드               0
총세대수               0
임대건물구분             0
지역                 0
공급유형               0
전용면적               0
전용면적별세대수           0
공가수                0
자격유형               0
임대보증금            177
임대료              177
지하철                0
버스정류장              0
단지내주차면수            0
주차면수/총세대수          0
공가수/주차면수           0
총임대세대수             0
qcut_총세대수          0
qcut_단지내주차면수       0
qcut_전용면적별세대수      0
10_전용면적            0
10평_비율             0
20평_비율             0
30평_비율             0
40평_비율             0
50평_비율             0
60평_비율             0
70평_비율             0
80평_비율             0
90평_비율             0
100평_비율            0
dtype: int64

In [270]:
test[test['임대료'].isnull()]['공급유형'].value_counts()

임대상가    177
Name: 공급유형, dtype: int64

### test 상가 결측치 처리

In [278]:
X = train1[train1['자격유형']=='A'].copy().reset_index(drop=True)

test_X = test1[test1['자격유형']=='D'].copy().reset_index(drop=True)

In [279]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '지역', '전용면적', '전용면적별세대수', '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)']

encoder = LabelEncoder()
encoder.fit(X['지역'])
X['지역'] = encoder.transform(X['지역'])
test_X['지역'] = encoder.transform(test_X['지역'])
print(encoder.classes_)

X_train = X[sel]
y_train = X['임대보증금']
X_test = test_X[sel]

['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']


In [280]:
## 다른 지역의 임대보증금 / 임대료 데이터를 실수형으로 변환
## 그 후 로그를 씌워줌
y_train = y_train.astype('float')
# y_train = y_train.astype('float')

y_train = np.log1p(y_train)
# y_train = np.log1p(y_train)

In [274]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [281]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

score = cross_val_score(LGBM, X_train, y_train, cv=5, 
                       scoring='neg_mean_absolute_error')

m_score = round(np.abs(score.mean()), 6)
print('LightGBM Score :', m_score)

pred = LGBM.predict(X_test)
pred[0:10]

LightGBM Score : 0.218203


array([16.11354143, 16.11354143, 17.10870058, 16.60871114, 16.60871114,
       16.60871114, 16.60871114, 16.60871114, 16.60871114, 16.60871114])

In [282]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C1006,NaN,NaN,9.954560e+06
1,C1006,NaN,NaN,9.954560e+06
2,C1006,NaN,NaN,2.692863e+07
3,C1006,NaN,NaN,1.633321e+07
4,C1006,NaN,NaN,1.633321e+07
5,C1006,NaN,NaN,1.633321e+07
6,C1006,NaN,NaN,1.633321e+07
7,C1006,NaN,NaN,1.633321e+07
8,C1006,NaN,NaN,1.633321e+07
9,C1006,NaN,NaN,1.633321e+07


### 임대보증금

In [283]:
null_idx = test[test['자격유형']=='D'].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대보증금'] = one
    
test.isnull().sum()

단지코드             0
총세대수             0
임대건물구분           0
지역               0
공급유형             0
전용면적             0
전용면적별세대수         0
공가수              0
자격유형             0
임대보증금            0
임대료              0
지하철              0
버스정류장            0
단지내주차면수          0
주차면수/총세대수        0
공가수/주차면수         0
총임대세대수           0
qcut_총세대수        0
qcut_단지내주차면수     0
qcut_전용면적별세대수    0
10_전용면적          0
10평_비율           0
20평_비율           0
30평_비율           0
40평_비율           0
50평_비율           0
60평_비율           0
70평_비율           0
80평_비율           0
90평_비율           0
100평_비율          0
dtype: int64

### 임대료

In [276]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = np.expm1(pred)

result_df

,단지코드,임대보증금,임대료,결과_임대료
0,C1006,NaN,NaN,148571.192347
1,C1006,NaN,NaN,148571.192347
2,C1006,NaN,NaN,197306.398937
3,C1006,NaN,NaN,186948.221029
4,C1006,NaN,NaN,186948.221029
5,C1006,NaN,NaN,186948.221029
6,C1006,NaN,NaN,186948.221029
7,C1006,NaN,NaN,186948.221029
8,C1006,NaN,NaN,186948.221029
9,C1006,NaN,NaN,186948.221029


In [277]:
null_idx = test[test['자격유형']=='D'].index
pred = np.expm1(pred)

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대료'] = one
    
test.isnull().sum()

단지코드               0
총세대수               0
임대건물구분             0
지역                 0
공급유형               0
전용면적               0
전용면적별세대수           0
공가수                0
자격유형               0
임대보증금            177
임대료                0
지하철                0
버스정류장              0
단지내주차면수            0
주차면수/총세대수          0
공가수/주차면수           0
총임대세대수             0
qcut_총세대수          0
qcut_단지내주차면수       0
qcut_전용면적별세대수      0
10_전용면적            0
10평_비율             0
20평_비율             0
30평_비율             0
40평_비율             0
50평_비율             0
60평_비율             0
70평_비율             0
80평_비율             0
90평_비율             0
100평_비율            0
dtype: int64

### test '-' 결측치 처리

In [296]:
test[test['임대보증금']=='-']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,-,0.0,1.0,40.0,0.333333,0.22500,120,0,0,2,20.0,0.0,0.550000,0.450000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,-,-,0.0,1.0,40.0,0.333333,0.22500,120,0,0,2,30.0,0.0,0.550000,0.450000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1014,C1267,675,아파트,경상남도,행복주택,16.94,50,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,2,20.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1015,C1267,675,아파트,경상남도,행복주택,26.85,66,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,2,30.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1016,C1267,675,아파트,경상남도,행복주택,26.85,8,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,1,30.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,3,40.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0


In [358]:
X = train2.copy().reset_index(drop=True)
test_X = test2[test2['임대보증금']=='-'].copy().reset_index(drop=True)

In [359]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

sel = ['총세대수', '지역', '전용면적', '공급유형', '전용면적별세대수', '자격유형',
       '지하철', '버스정류장', '단지내주차면수', 'qcut_전용면적별세대수', 
      '20대(남자)', '30대(남자)', '40대(남자)', '50대(여자)']

encoder = LabelEncoder()
enco = ['지역', '공급유형', '자격유형']

for one in enco:
    encoder.fit(X[one])
    X[one] = encoder.transform(X[one])
    test_X[one] = encoder.transform(test_X[one])
    print(encoder.classes_)

X_train = X[sel]
y_train = X['임대료']
X_test = test_X[sel]

['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']


In [360]:
y_train = np.log1p(y_train)

In [361]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.05,
                   'min_child_samples': 20, 
                   'num_leaves': 56,
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 10000}

In [373]:
LGBM = lgb.LGBMRegressor(**hyperparameters)

LGBM.fit(X_train, y_train)

pred = np.expm1(LGBM.predict(X_test))
pred[0:10]

array([ 90565.22794043, 105439.15779959,  35104.93317036,  51724.00126796,
        52054.11819329,  82662.27754769])

In [355]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대보증금'] = pred

result_df

,단지코드,임대보증금,임대료,결과_임대보증금
0,C2152,-,-,9.313186e+06
1,C2152,-,-,1.206536e+07
2,C1267,-,-,1.853237e+07
3,C1267,-,-,2.422621e+07
4,C1267,-,-,2.683225e+07
5,C1267,-,-,3.050036e+07


### 임대보증금

In [365]:
test[test['임대료']=='-']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,9.31319e+06,-,0.0,1.0,40.0,0.333333,0.22500,120,0,0,2,20.0,0.0,0.550000,0.450000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,1.20654e+07,-,0.0,1.0,40.0,0.333333,0.22500,120,0,0,2,30.0,0.0,0.550000,0.450000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1014,C1267,675,아파트,경상남도,행복주택,16.94,50,38.0,L,1.85324e+07,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,2,20.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1015,C1267,675,아파트,경상남도,행복주택,26.85,66,38.0,L,2.42262e+07,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,2,30.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1016,C1267,675,아파트,경상남도,행복주택,26.85,8,38.0,L,2.68323e+07,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,1,30.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,3.05004e+07,-,0.0,1.0,467.0,0.691852,0.08137,670,2,2,3,40.0,0.0,0.432836,0.110448,0.18806,0.268657,0.0,0.0,0.0,0.0,0.0


In [379]:
test[test['단지코드']=='C2152']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,9.31319e+06,90565.2,0.0,1.0,40.0,0.333333,0.225,120,0,0,2,20.0,0.0,0.55,0.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,1.20654e+07,105439,0.0,1.0,40.0,0.333333,0.225,120,0,0,2,30.0,0.0,0.55,0.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [357]:
null_idx = test[test['임대보증금']=='-'].index

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대보증금'] = one
    
(test['임대보증금']=='-').sum()

0

### 임대료

In [375]:
result_df = test_X[['단지코드', '임대보증금', '임대료']].copy()
result_df['결과_임대료'] = pred

result_df

,단지코드,임대보증금,임대료,결과_임대료
0,C2152,-,-,90565.227940
1,C2152,-,-,105439.157800
2,C1267,-,-,35104.933170
3,C1267,-,-,51724.001268
4,C1267,-,-,52054.118193
5,C1267,-,-,82662.277548


In [378]:
null_idx = [1005, 1006, 1014, 1015, 1016, 1017]

for idx, one in zip(null_idx, pred):
    test.loc[idx, '임대료'] = one
    
(test['임대료']=='-').sum()

0

In [383]:
train['임대료'] = train['임대료'].astype('float')
train['임대보증금'] = train['임대보증금'].astype('float')

test['임대료'] = test['임대료'].astype('float')
test['임대보증금'] = test['임대보증금'].astype('float')

In [387]:
test.describe()

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
count,1022.000000,1022.000000,1022.000000,1022.000000,1.022000e+03,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.0,1022.000000
mean,862.080235,43.706311,100.414873,15.544031,2.292694e+07,166306.211039,0.131115,4.626223,548.771037,0.729714,0.038149,860.508806,1.973581,1.953033,1.977495,42.250489,0.000248,0.058816,0.262887,0.289739,0.301527,0.063443,0.005711,0.017422,0.0,0.000207
std,536.340894,35.890759,125.997855,11.070140,1.608795e+07,95479.680462,0.427313,5.414568,342.636703,0.379583,0.046539,537.216858,1.397594,1.433387,1.419910,13.692577,0.001138,0.139150,0.367788,0.232243,0.290469,0.143727,0.046969,0.120390,0.0,0.000461
min,75.000000,9.960000,1.000000,0.000000,2.249000e+06,17420.000000,0.000000,1.000000,29.000000,0.149009,0.000000,70.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,488.000000,33.135000,14.000000,6.000000,1.200950e+07,100845.000000,0.000000,2.000000,286.000000,0.691906,0.009892,482.000000,1.000000,1.000000,1.000000,30.000000,0.000000,0.000000,0.000000,0.005443,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,745.000000,39.720000,60.000000,15.000000,1.876050e+07,142385.000000,0.000000,3.000000,458.000000,0.807910,0.026991,745.000000,2.000000,2.000000,2.000000,40.000000,0.000000,0.000000,0.048091,0.327679,0.268657,0.000000,0.000000,0.000000,0.0,0.000000
75%,1161.000000,47.400000,140.000000,23.000000,2.872300e+07,207900.000000,0.000000,5.000000,711.000000,0.898080,0.052770,1161.000000,3.000000,3.000000,3.000000,50.000000,0.000000,0.006029,0.414388,0.472089,0.529338,0.071082,0.000000,0.000000,0.0,0.000000
max,2572.000000,583.400000,1341.000000,45.000000,1.444960e+08,580000.000000,2.000000,50.000000,1696.000000,5.678571,0.413793,2572.000000,4.000000,4.000000,4.000000,100.000000,0.006938,0.750000,0.990000,1.000000,1.000000,1.000000,0.644809,1.000000,0.0,0.001982


In [388]:
train.to_csv('fill_na_train.csv', index=False)
test.to_csv('fill_na_test.csv', index=False)

import os

os.listdir(os.getcwd())

['.ipynb_checkpoints',
 '07_17_first_submit.csv',
 '07_17_second_submit.csv',
 '07_17_third_submit.csv',
 '07_18_first_submit.csv',
 '07_18_second_submit.csv',
 '07_18_third_submit.csv',
 '07_19_first_submit.csv',
 '07_19_second_submit.csv',
 '07_19_third_submit.csv',
 '07_20_first_submit.csv',
 '07_20_second_submit.csv',
 '07_20_third_submit_pred1.csv',
 '07_23_first_submit.csv',
 '07_23_second_submit.csv',
 '07_23_third_submit.csv',
 '07_24_SS_first_submit.csv',
 '07_24_SS_second_submit.csv',
 '07_24_SS_third_submit.csv',
 '210721_Baseline_RYU.ipynb',
 '3조_Baseline.ipynb',
 'CSV',
 'fill_na_test.csv',
 'fill_na_train.csv',
 'parking_data_preprocessing_test_07_17.csv',
 'parking_data_preprocessing_test_07_18.csv',
 'parking_data_preprocessing_test_07_19.csv',
 'parking_data_preprocessing_test_07_20.csv',
 'parking_data_preprocessing_train_07_17.csv',
 'parking_data_preprocessing_train_07_18.csv',
 'parking_data_preprocessing_train_07_19.csv',
 'parking_data_preprocessing_train_07_20.c

In [286]:
pd.set_option('display.max_rows', 30)

train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,4,30.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,2,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,3,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0


### Age 정보 넣기

In [344]:
## train1, test1 : 라벨인코딩 + age정보 같이 된 데이터프레임
train2 = train.copy()
test2 = test.copy()

# age.index = age['지역']
# age.drop('지역', axis=1, inplace=True)

In [345]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])

for i in range(train.shape[0]):
    area = train.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
    
age_df

,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
3,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
4,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2865,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2866,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2867,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


In [346]:
train2 = pd.concat([train2, age_df], axis=1)
train2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000.0,82940,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,4,30.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000.0,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,2,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000.0,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,40.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000.0,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000.0,149760,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,1,50.0,0.0,0.0,0.506422,0.146789,0.346789,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000.0,116090,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000.0,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000.0,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,1,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000.0,142310,0.0,1.0,166.0,146.0,0.694561,0.042169,239,0,0,3,50.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,0.068002,0.087266,0.072177,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


In [347]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])

for i in range(test.shape[0]):
    area = test.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
    
age_df

,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
2,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
3,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1018,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1019,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1020,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


In [348]:
test2 = pd.concat([test2, age_df], axis=1)
test2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,10_전용면적,10평_비율,20평_비율,30평_비율,40평_비율,50평_비율,60평_비율,70평_비율,80평_비율,90평_비율,100평_비율,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,40.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,1,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754,2,3,3,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,0.905836,0.020498,754,2,3,2,50.0,0.0,0.000000,0.000000,0.153846,0.846154,0.0,0.0,0.0,0.0,0.0,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.081370,670,2,2,3,40.0,0.0,0.432836,0.110448,0.188060,0.268657,0.0,0.0,0.0,0.0,0.0,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378,0,1,3,30.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378,0,1,1,30.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0,0.785340,0.150000,378,0,1,4,40.0,0.0,0.000000,0.306878,0.534392,0.158730,0.0,0.0,0.0,0.0,0.0,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


In [382]:
train2['임대료'] = train2['임대료'].astype('float')
train2['임대보증금'] = train2['임대보증금'].astype('float')

test2['임대료'] = test2['임대료'].astype('float')
test2['임대보증금'] = test2['임대보증금'].astype('float')

ValueError: could not convert string to float: '-'

In [390]:
train.corr()['임대보증금'].sort_values()

30평_비율          -0.275553
100평_비율         -0.259951
10평_비율          -0.178640
40평_비율          -0.075181
공가수/주차면수        -0.043910
전용면적별세대수         0.008930
총임대세대수           0.012164
지하철              0.016980
총세대수             0.024668
50평_비율           0.041036
qcut_총세대수        0.057960
20평_비율           0.100502
버스정류장            0.108067
qcut_전용면적별세대수    0.111201
공가수              0.121783
60평_비율           0.209671
70평_비율           0.296849
단지내주차면수          0.309759
qcut_단지내주차면수     0.325332
전용면적             0.342594
80평_비율           0.342794
등록차량수            0.352220
주차면수/총세대수        0.375599
임대료              0.530520
10_전용면적          0.541955
임대보증금            1.000000
90평_비율                NaN
Name: 임대보증금, dtype: float64

### LabelEncoding

In [30]:
## 우선 모두 진행
from sklearn.preprocessing import LabelEncoder

cate = ['임대건물구분', '지역', '자격유형', '공급유형']

encoder = LabelEncoder()

for one in cate:
    encoder.fit(train1[one])
    train1[one] = encoder.transform(train1[one])
    print(encoder.classes_)
    
    test1[one] = encoder.transform(test1[one])
    print(encoder.classes_)

['상가' '아파트']
['상가' '아파트']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['강원도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '전라남도' '전라북도' '제주특별자치도' '충청남도' '충청북도']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']
['공공분양' '공공임대(10년)' '공공임대(50년)' '공공임대(5년)' '공공임대(분납)' '국민임대' '영구임대' '임대상가'
 '장기전세' '행복주택']


In [39]:
print(train1.columns)
print()
print(test1.columns)

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '등록차량수', '주차면수/총세대수',
       '공가수/주차면수', '총임대세대수', 'qcut_총세대수', 'qcut_단지내주차면수', 'qcut_전용면적별세대수',
       '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)', '10_전용면적', '10평_비율',
       '20평_비율', '30평_비율', '40평_비율', '50평_비율', '60평_비율', '70평_비율', '80평_비율',
       '90평_비율', '100평_비율'],
      dtype='object')

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '주차면수/총세대수',
       '공가수/주차면수', '총임대세대수', 'qcut_총세대수', 'qcut_단지내주차면수', 'qcut_전용면적별세대수',
       '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)